# Packages

In [1]:
from ipywidgets import HTML
import ipywidgets as widgets
from ipyleaflet import Map, Polyline, Rectangle, basemaps, basemap_to_tiles, Polygon, FullScreenControl, Popup, WidgetControl
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Functions

In [5]:
def rotate(point, rot):
    return (point[0]*math.cos(math.radians(rot)) - point[1]*math.sin(math.radians(rot)), 
            point[0]*math.sin(math.radians(rot)) + point[1]*math.cos(math.radians(rot)))

def get_bounds(breadth, length, longitude, latitude, rot_angle):
    #This function gets the real bounds of a boat
    #Knowing that according to: https://en.wikipedia.org/wiki/Decimal_degrees
    #0.00001deg are equal to 1.1132 m
    length = length*0.00001/1.1132
    breadth = breadth*0.00001/1.1132
    
    center_point = np.array((longitude, latitude))
    
    ship_shape_ini = [(-breadth/2, length/2), (breadth/2, length/2), (breadth/2, -length/2), (-breadth/2, -length/2)]
    
    ship_shape_rot = [np.array(rotate(point, rot_angle)) for point in ship_shape_ini]
    
    xy1 = list(ship_shape_rot[0] + center_point)
    xy2 = list(ship_shape_rot[1] + center_point)
    xy3 = list(ship_shape_rot[2] + center_point)
    xy4 = list(ship_shape_rot[3] + center_point)
    
    return [xy1,xy2,xy3,xy4]

def get_color_scale(arr, scale_type='plasma'):
    arr = list(arr)
    #https://matplotlib.org/gallery/color/colormap_reference.html
    scale = cm.get_cmap(scale_type, len(arr)).colors
    
    arr = np.array(arr)
    sorted_index = np.argsort(arr)
    return {arr[sorted_index[i]]:rgb_to_hex(scale[i]) for i in range(len(arr))}

def rgb_to_hex(rgb):
    if len(rgb) == 4:
        rgb = rgb[:3]
    return '#%02x%02x%02x' % tuple(list([int(elem*256) for elem in rgb]))
def random_hex_color():
    import random
    r = lambda: random.randint(0,255)
    return ('#%02X%02X%02X' % (r(),r(),r()))

# Loading DataSet

In [6]:
filename = '/datc/saab/ANONIMISED_trackdata_small_export_HH_proj_anomalydetection.csv'
"""data = pd.DataFrame()

row_count = 1000
i = 0
for chunk in pd.read_csv(filename, chunksize=row_count):
    print('CHUNK ', i)
    data = pd.concat([data, chunk])
    i+=1"""

data = pd.read_csv(filename)

data.latitude = data['latitude'] + 47.72
data.longitude = data['longitude'] + 157.85
data

,mmsi,datetime,latitude,longitude,orientation,rateofturn,course,length,breadth,speed,vesseltype
0,NaN,2018-11-30 16:00:00.116,22.172527,113.636237,77.5,NaN,81.5,24.3,3.0,21.61,NaN
1,NaN,2018-11-30 16:00:00.206,22.372892,113.913979,147.2,NaN,142.3,32.5,4.8,4.84,NaN
2,290316673.0,2018-11-30 16:00:00.206,22.366451,113.914867,317.0,3.4,317.5,50.0,15.8,2.97,8.0
3,NaN,2018-11-30 16:00:00.206,22.371795,113.913800,105.8,NaN,197.4,31.6,5.6,0.01,NaN
4,NaN,2018-11-30 16:00:00.206,22.357411,113.930053,289.7,NaN,294.0,17.8,2.3,2.79,NaN
...,...,...,...,...,...,...,...,...,...,...,...
42501864,289025527.0,2018-12-01 15:41:12.707,21.855550,114.140983,0.0,NaN,158.4,26.0,7.0,4.17,0.0
42501865,NaN,2018-12-01 15:54:03.707,22.158087,114.148156,209.6,-6.9,209.4,35.7,5.1,1.93,NaN
42501866,NaN,2018-12-01 15:55:59.206,22.350758,113.795325,96.3,10.3,119.0,50.0,14.8,3.36,NaN
42501867,NaN,2018-12-01 15:55:59.206,22.483589,113.793135,304.3,NaN,30.2,23.1,3.6,0.64,NaN


## Visualize the 5 first rows of data

In [7]:
print(set([elem[:10] for elem in data[' datetime'].unique()]))
data = data.dropna()

{'2018-11-30', '2018-12-01'}


# Data visualization (map)

### Define variables of the visualization

In [8]:
number_of_ships = 1000


CRAFT_length_list = data.length.unique()
color_scale = get_color_scale(CRAFT_length_list, )

CRAFT_ID_list = data.mmsi.unique()#Get the mmsi unique values into a list:


ships_info = [[[] for  i in range(31)] for j in range(number_of_ships)]#List that will storage a list of lists == a list of time-series(which will as well be represented as a list)

ship_number = 0
for rowid in CRAFT_ID_list[:number_of_ships]:
    #Start with empty lists
    npinfo, infolist = [], []
    #Get a numpy array composed by 'latitude', 'longitude', 'orientation', 'length', 'breadth'
    npinfo = data[data.mmsi == rowid][['latitude', 'longitude', 'orientation', 'length', 'breadth', ' datetime']].values
    #Convert it to a python list so it can be an attribute of the multypoligon functionality of ipyleaflet
    infolist = coordslist = list([list(coords) for coords in npinfo])
    for point in infolist:
        ships_info[ship_number][int(point[5].split('-')[2][:2]) - 1].append(point)
    ship_number+=1

In [9]:
print(ships_info[3][29])

[[22.352762284, 113.99319762799999, 97.6, 35.0, 4.7, '2018-11-30 16:00:00.206'], [22.35274925, 113.993285052, 95.9, 35.2, 4.7, '2018-11-30 16:00:03.207'], [22.352742838, 113.993370379, 96.9, 35.4, 4.7, '2018-11-30 16:00:06.207'], [22.352730264999998, 113.99346165799999, 97.9, 35.5, 4.7, '2018-11-30 16:00:09.207'], [22.352717189, 113.993546735, 98.5, 35.6, 4.8, '2018-11-30 16:00:12.207'], [22.352701891999995, 113.993642624, 97.8, 35.8, 4.9, '2018-11-30 16:00:15.206'], [22.352684541, 113.99372468199999, 98.9, 36.0, 4.9, '2018-11-30 16:00:18.207'], [22.352671255999997, 113.99381294399998, 99.5, 36.1, 4.9, '2018-11-30 16:00:21.207'], [22.35264611, 113.993896847, 100.4, 36.3, 5.0, '2018-11-30 16:00:24.207'], [22.352620251999998, 113.993978654, 101.2, 36.5, 5.0, '2018-11-30 16:00:27.206'], [22.352601393, 113.99406691600001, 101.4, 36.6, 5.0, '2018-11-30 16:00:30.207'], [22.352572308, 113.99414830399999, 102.2, 36.8, 5.0, '2018-11-30 16:00:33.207'], [22.35255349, 113.99423011100001, 102.2, 37

### Visualize

In [18]:
m = Map(center = (22.205232, 114.123882), zoom = 12)#Define the map object

#To define  the maximum number of steps we will be able to take with the slider
max_steps = max([len(element) for element in ships_info])
ships_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=200,
    step=1,
    description='Ships: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
widget_steps = WidgetControl(widget=ships_slider, position='topright')
m.add_control(widget_steps)
m.add_control(FullScreenControl())
dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
m.add_layer(dark_matter_layer)
previous_value = 0


def update_map(ships_slider):
    global previous_value, m
    if previous_value > ships_slider:
        m = Map(center = (22.205232, 114.123882), zoom = 12)#Define the map object
        ini, end = 0, ships_slider
    else:
        ini, end = previous_value, ships_slider
        
        
    for i in range(ini, end):
        color_value = random_hex_color()
        #for each time series in ships_info list --> Paint The line
        line = Polyline(
            locations = [[elem[0],elem[1]] for elem in ships_info[i][29]],
            color = color_value,
            fill_color= "transparent",
            weight = 3,
            opacity = 0.3)

        m.add_layer(line)
        if ships_info[i][29] != []:
            singular_info = ships_info[i][29][-1]
            message = HTML()
            message.value = '<b>**DATE: <br>' + str(singular_info[-1]) + '**</b><br><b>Longitude</b>: ' + str(singular_info[0]) + '<br><b>Latitude</b>:  ' + str(singular_info[1]) + "<br><b>Vessel's Length</b>: " + str(singular_info[3]) + "<br><b>Vessel's Breadth</b>: " + str(singular_info[4]) + "<br><b>Facing:</b>: " + str(singular_info[2]) + ' deg'

            mapbounds = get_bounds(singular_info[3], singular_info[4], singular_info[0], singular_info[1], singular_info[2])
            ship_shape = Polygon(
            locations=[mapbounds],
            color=color_value,
            fill_color=color_value,
            fill_opacity=1
            )
            ship_shape.popup = message
            m.add_layer(ship_shape)
    previous_value = ships_slider
display(m)
widgets.interactive(update_map, ships_slider=ships_slider)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

interactive(children=(IntSlider(value=0, continuous_update=False, description='Ships: ', max=200), Output()), …